# Extract Features from SCFace images with ArcFace

Images are processed with default model from arcface. Information of identity, camera and distance are also saved.

In [1]:
import insightface
import cv2
import numpy as np
import pickle
import os
import time
import pandas as pd
os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT']='0'

In [2]:
cwd = os.getcwd()
img_folder = '/media/rafael/Windows-SSD/recfac/bases/SCFace/SCface_database/mugshot_frontal_cropped_all'

In [3]:
# Initialize model
model = insightface.app.FaceAnalysis(ga_name=None)

# set to -1 for CPU or positive for # of GPUs
ctx_id = 0

model.prepare(ctx_id = ctx_id, nms=0.4)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [4]:
def enroll(imgPath):
    rgbImg = cv2.cvtColor(cv2.imread(imgPath),cv2.COLOR_BGR2RGB)
    if rgbImg.shape[0] > 400 or rgbImg.shape[1] > 400:
        ratio = 400./rgbImg.shape[0]
        rgbImg = cv2.resize(rgbImg, None, fx = ratio, fy = ratio)
    center = np.array([int(rgbImg.shape[0]/2),int(rgbImg.shape[1]/2)])
    faces = model.get(rgbImg)
    dist=[]
    rep = np.zeros(512)
    img_name = os.path.splitext(os.path.basename(imgPath))[0]
    identity = img_name[0:3]
    
    # Indicate if no face is detected
    if len(faces) == 0:
        print("No face found on {}".format(imgPath))
        rep[:] = np.nan
        status = "no face"
        return status, identity, rep
    
    # Compute centroids of faces and distances from certer of image
    for face in faces:
        box=face.bbox.astype(np.int).flatten()
        centroid = np.array([int((box[0]+box[2])/2),int((box[1]+box[3])/2)])
        dist.append(np.linalg.norm(center-centroid))
    
    # Get embeddings of the face with centroid closest to the center of the image
    idx_face = dist.index(min(dist))
    rep = faces[idx_face].embedding
    
    status = "ok"    
    return status, identity, rep

In [ ]:
# Initialize the template dictionary
templates = {}
print ("Enrolling faces.")
start_time = time.time()

# Get list of images to enroll
image_list = os.listdir(img_folder)
number_of_images = len(image_list)

# Loop through folder and extract the embedding for central face of each image
for i,img in enumerate(image_list):
    # Get the embedding for each face and store it in a template dictionary
    print("Processing file {}. ({}/{})".format(img,i+1,number_of_images),end='\r')
    imgPath = os.path.join(img_folder , img)
    templates[img] = enroll(imgPath)

# Write templates to gallery file
print ("\nEnrolling {} files took {:0.2f} seconds.".format(len(templates), time.time() - start_time))

In [6]:
#Create gallery file and write templates dictionary to it
with open('scface_mugshots_cropped_400.gal',"wb") as gallery:
    pickle.dump(templates, gallery)

In [8]:
features_df = pd.DataFrame.from_dict(templates, orient='index', columns=['status','id','features'])
features_df.head()

,status,id,features
001_frontal.JPG,ok,001,"[-0.2964732, 0.10427113, -0.64649826, 0.749349..."
002_frontal.JPG,ok,002,"[0.1382779, 1.695791, 0.2079391, -2.1961703, -..."
003_frontal.JPG,ok,003,"[1.449193, -0.69684964, 0.1824857, -0.04981476..."
004_frontal.JPG,ok,004,"[0.4943828, 0.43095377, 0.39696738, 0.16549608..."
005_frontal.JPG,ok,005,"[1.0114638, 0.54829, 0.25893757, 0.69848627, 0..."


In [11]:
#check how many images failed to enroll
features_df.loc[features_df['status'] == 'no face'].groupby(['id']).count()['features']

Series([], Name: features, dtype: int64)

In [12]:
#remove images that failed to enroll (status = 'no face')
features_df = features_df[features_df.status == 'ok']
len(features_df)

130

In [13]:
ids = list(features_df['id'].unique())
ids.sort()
len(ids)

130

In [15]:
feat1, feat2 = features_df.loc['001_frontal.JPG']['features'], features_df.loc['002_frontal.JPG']['features']
np.dot(feat1, feat2)/(np.linalg.norm(feat1)*np.linalg.norm(feat2))

0.039551493